## optimize hyperparameters with Optuna

### load data en preprocess

In [1]:
import sys

sys.path.append("../../")

import utils.load_csv as load_csv

set = load_csv.load_pandas()
set = load_csv.sample_data(set, 100)

# train and val only contain inliers and test contains both inliers and outliers
train, val, test = load_csv.split_data(set)
print(f"train length: {len(train)}")
print(f"val length: {len(val)}")
print(f"test length: {len(test)}")

print(f'outlier test training: {any(train["classification"] != 1)}')
print(f'outlier test validation: {any(val["classification"] != 1)}')
print(f'outlier test test: {any(test["classification"] != 1)}')
train

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/VAE/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


train length: 45
val length: 15
test length: 18
outlier test training: False
outlier test validation: False
outlier test test: True


,classification,file_names,label
2894,1,/home/bendm/machine_learning/project/majority-...,Singlet
2876,1,/home/bendm/machine_learning/project/majority-...,Singlet
4947,1,/home/bendm/machine_learning/project/majority-...,Singlet
5861,1,/home/bendm/machine_learning/project/majority-...,Singlet
2678,1,/home/bendm/machine_learning/project/majority-...,Singlet
7231,1,/home/bendm/machine_learning/project/majority-...,Singlet
9930,1,/home/bendm/machine_learning/project/majority-...,Singlet
971,1,/home/bendm/machine_learning/project/majority-...,Singlet
6340,1,/home/bendm/machine_learning/project/majority-...,Singlet
4640,1,/home/bendm/machine_learning/project/majority-...,Singlet


In [2]:
from models.VAE.VAE import VAE
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter
from utils.dataloader import ImagePathDataset
from torchvision.transforms import v2

In [3]:
import torch.optim as optim
import optuna


def objective(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_dim = 4800
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    hidden_dim = trial.suggest_int("hidden_dim", 1000, 4000, step=500)
    latent_dim = trial.suggest_int("latent_dim", 10, 1000, step=10)
    epochs = trial.suggest_int("epochs", 10, 100)

    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    #learning_rate = 1e-3
    weight_decay = 1e-2

    model = VAE(input_dim, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=learning_rate, weight_decay=weight_decay
    )

    transform = v2.Compose(
        [
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Resize((60, 80)),
            v2.ToTensor(),
            v2.Lambda(
                lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
            ),
        ]
    )

    train_set = ImagePathDataset(train, transform=transform)
    val_set = ImagePathDataset(val, transform=transform)

    train_dataloader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True
    )
    val_dataloader = torch.utils.data.DataLoader(
        val_set, batch_size=batch_size, shuffle=True
    )

    prev_updates = 0
    best_val_loss = float("inf")
    patience, early_stop_counter = 5, 0

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        prev_updates = model.train_model(train_dataloader, optimizer, prev_updates)
        val_loss = model.test(val_dataloader, prev_updates)
        best_val_loss = min(val_loss, best_val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping...")
                break

        # Optional pruning
        trial.report(best_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss

In [4]:
study = optuna.create_study(direction="minimize", 
                            study_name="VAE",
                            sampler=optuna.samplers.TPESampler(),
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=30))
study.optimize(objective, n_trials=100, timeout=600)


[I 2024-11-14 16:47:27,754] A new study created in memory with name: VAE
/tmp/ipykernel_132500/3080735280.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
/home/bendm/machine_learning/project/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Epoch 1/92


  0%|          | 0/1 [00:00<?, ?it/s]

Step 0 (N samples: 0), Loss: 3333.9170 (Recon: 3327.4785, KL: 6.4384) Grad: 27.9591


Testing: 100%|██████████| 1/1 [00:00<00:00, 15.69it/s]


...

====> Test set loss: 4150.5713 (BCE: 4093.0017, KLD: 57.5697)
Epoch 4/100


Testing: 100%|██████████| 1/1 [00:00<00:00, 39.13it/s]


====> Test set loss: 2111.8574 (BCE: 2106.0417, KLD: 5.8157)
Epoch 5/100


Testing: 100%|██████████| 1/1 [00:00<00:00, 34.81it/s]
[I 2024-11-14 16:51:44,102] Trial 99 finished with value: 2036.510986328125 and parameters: {'batch_size': 16, 'hidden_dim': 1000, 'latent_dim': 70, 'epochs': 100, 'learning_rate': 0.0016597345763282672}. Best is trial 47 with value: 1993.71728515625.


====> Test set loss: 2036.5110 (BCE: 2029.0332, KLD: 7.4778)
Early stopping...


In [5]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Number of finished trials:  100
Best trial:
  Value: 1993.71728515625
  Params: 
    batch_size: 16
    hidden_dim: 2000
    latent_dim: 30
    epochs: 77
    learning_rate: 0.0013039505192832476


In [6]:
import optuna.visualization as vis

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_intermediate_values(study).show()